In [1]:
!pip install transformers
!pip install libwapiti parsivar
!pip show libwapiti
!pip install tensorflow
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.8 MB/s 
     |████████████████████████████████| 6.6 MB 37.1 MB/s 
     |████████████████████████████████| 596 kB 43.5 MB/s 
     |████████████████████████████████| 86 kB 1.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 233 kB 8.0 MB/s 
     |████████████████████████████████| 36.2 MB 1.1 MB/s 
     |████████████████████████████████| 1.5 MB 41.3 MB/s 
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154825 sha256=de6c8ea14e52b36738daa849a32de17a31a2bb78dd0c5e6a22559f9264f1ab1d
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a

In [2]:
import pandas as pd
from transformers import MarianMTModel, MarianConfig
from transformers import MarianTokenizer
from parsivar import *
import pprint
from tokenizers import Tokenizer
from tokenizers.models import BPE

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
tokenizer = Tokenizer(BPE())

In [5]:
#آوردن دیتا از درایو
!cp /content/drive/MyDrive/NLP/CA05/data.zip /content/data.zip
!unzip /content/data.zip

Archive:  /content/data.zip
   creating: data/
  inflating: data/test.en            
  inflating: data/test.fa            
  inflating: data/train.en           
  inflating: data/train.fa           
  inflating: data/valid.en           
  inflating: data/valid.fa           


In [6]:
!cp /content/drive/MyDrive/NLP/CA05/spell.zip /content/spell.zip
!unzip spell.zip
!mkdir /usr/local/lib/python3.7/dist-packages/parsivar/resource/spell
!mv /content/mybigram_lm.pckl /usr/local/lib/python3.7/dist-packages/parsivar/resource/spell
!mv /content/onegram.pckl /usr/local/lib/python3.7/dist-packages/parsivar/resource/spell
#کپی کردن فایل های مورد نیاز برای تصحیح متن فارسی

Archive:  spell.zip
  inflating: mybigram_lm.pckl        
  inflating: onegram.pckl            


In [7]:
!pip install autocorrect #کتاب خانه تصحیح متن انگلیسی

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 622 kB 7.2 MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622382 sha256=d8e8400db9a49eeaa1cf6b5b6e9e7c9a5a87873c5819250d6fe240f6367589d5
  Stored in directory: /root/.cache/pip/wheels/54/d4/37/8244101ad50b0f7d9bffd93ce58ed7991ee1753b290923934b
Successfully built autocorrect


In [8]:
from parsivar import SpellCheck
from autocorrect import Speller
spell = Speller(lang='en')
spcheck = SpellCheck()

In [9]:
def perNormalizer(sentence):
    sentence = sentence.replace('ي', 'ی')
    sentence = sentence.replace('ك', 'ک')
    sentence = sentence.replace('ٔ', '')
    sentence = spcheck.spell_corrector(sentence)
    return sentence

In [ ]:
spell('caaar')

'car'

# New Section

In [10]:
#تابع پیش پردازش فایل ها
def load_and_pre_processing(inputPath, outputPath, per=False):
  f = open(inputPath,'r')
  fileText = f.readlines()
  if per:
    fileOut = [perNormalizer(text) + '\n' for text in fileText]
  else:
    fileOut = [spell(text) for text in fileText]
  with open(outputPath,'w') as f:
    f.writelines(fileOut)



In [11]:
#پیش پردازش فایل ها
load_and_pre_processing('data/test.en', 'data/test.en2')

In [12]:
load_and_pre_processing('data/test.fa', 'data/test.fa2', per=True)

In [13]:
load_and_pre_processing('data/train.en', 'data/train.en2')
load_and_pre_processing('data/train.fa', 'data/train.fa2', per=True)

In [14]:
load_and_pre_processing('data/valid.en', 'data/valid.en2')
load_and_pre_processing('data/valid.fa', 'data/valid.fa2', per=True)

In [ ]:
text_files = ['test.en', 'test.fa', 'train.en', 'train.fa', 'valid.en', 'valid.fa']
for fname in text_files:
  f = open('data/'+fname)
  tlist = f.readlines()
  tlist = [tokenizer.encode(x) for x in tlist]

In [24]:
#آموزش دادن توکنایزر
!python /content/OpenNMT-tf/third_party/learn_bpe.py -i data/train.en2 -o data/en.code -s 10000

In [25]:
!python /content/OpenNMT-tf/third_party/learn_bpe.py -i data/train.fa2 -o data/fa.code -s 10000

In [26]:
#اعمال توکنایزر
!python /content/OpenNMT-tf/third_party/apply_bpe.py -c data/en.code -i data/train.en2 -o data/train.en.bpe
!python /content/OpenNMT-tf/third_party/apply_bpe.py -c data/en.code -i data/valid.en2 -o data/valid.en.bpe

In [27]:
!python /content/OpenNMT-tf/third_party/apply_bpe.py -c data/en.code -i data/test.en2 -o data/test.en.bpe

In [28]:
!python /content/OpenNMT-tf/third_party/apply_bpe.py -c data/fa.code -i data/train.fa2 -o data/train.fa.bpe
!python /content/OpenNMT-tf/third_party/apply_bpe.py -c data/fa.code -i data/valid.fa2 -o data/valid.fa.bpe

In [20]:
#تابع برای لود دیتافریم و نمایش آن در خروجی
def loadDF(englishPath,persianPath):
  f = open(englishPath,'r')
  eng = f.readlines()
  f.close()
  f = open(persianPath,'r')
  per = f.readlines()
  f.close()
  print('eng: ',len(eng))
  print('per: ',len(per))
  return pd.DataFrame.from_dict({'eng':eng, 'per':per})

In [29]:
trainDf2 = loadDF('/content/data/train.en', '/content/data/train.fa')
trainDf = loadDF('/content/data/train.en.bpe', '/content/data/train.fa.bpe')
validDf = loadDF('/content/data/valid.en.bpe', '/content/data/valid.fa.bpe')
# testDf = loadDF('/content/data/test.en.bpe', '/content/data/test.fa.bpe')

eng:  30000
per:  30000
eng:  30000
per:  30000
eng:  400
per:  400


In [ ]:
trainDf2.head(10).style.hide_index()

In [ ]:
trainDf.head(10).style.hide_index()

In [ ]:
trainDf.style.hide_index()

In [30]:
#ذخیره فایل خواسته شده در مورد بعد از پیش پردازش
trainDf.to_csv('trainDf.csv',index=False)

# openNMT

In [ ]:
!cp /content/drive/MyDrive/NLP/CA05/data.zip /content/data.zip
!unzip /content/data.zip

Archive:  /content/data.zip
   creating: data/
  inflating: data/test.en            
  inflating: data/test.fa            
  inflating: data/train.en           
  inflating: data/train.fa           
  inflating: data/valid.en           
  inflating: data/valid.fa           


In [22]:
!git clone -b r1 https://github.com/OpenNMT/OpenNMT-tf.git

Cloning into 'OpenNMT-tf'...
remote: Enumerating objects: 34424, done.
remote: Counting objects: 100% (6084/6084), done.
remote: Compressing objects: 100% (792/792), done.
remote: Total 34424 (delta 5304), reused 5914 (delta 5230), pack-reused 28340
Receiving objects: 100% (34424/34424), 23.44 MiB | 23.67 MiB/s, done.
Resolving deltas: 100% (28966/28966), done.


In [23]:
!pip install OpenNMT-tf[tensorflow_gpu]==1.*

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 150 kB 7.2 MB/s 
     |████████████████████████████████| 16.6 MB 46.4 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 411.0 MB 17 kB/s 
     |████████████████████████████████| 3.8 MB 41.3 MB/s 
     |████████████████████████████████| 2.9 MB 35.7 MB/s 
     |████████████████████████████████| 20.1 MB 488 kB/s 
     |████████████████████████████████| 503 kB 59.0 MB/s 
     |████████████████████████████████| 50 kB 5.1 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=55324e1040185a676127c1fa4a89ca7e6dbafe51822703889ab9fd06725ca64c
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:


In [ ]:
!python OpenNMT-tf/third_party/learn_bpe.py -i data/train.en -o data/en.code -s 10000

In [ ]:
!python OpenNMT-tf/third_party/learn_bpe.py -i data/train.fa -o data/fa.code -s 10000

In [ ]:
!python OpenNMT-tf/third_party/apply_bpe.py -c  data/en.code -i  /content/data/train.en -o /content/data/train.en.bpe
!python OpenNMT-tf/third_party/apply_bpe.py -c  data/en.code -i  /content/data/valid.en -o /content/data/valid.en.bpe
!python OpenNMT-tf/third_party/apply_bpe.py -c data/en.code -i /content/data/test.en -o /content/data/test.en.bpe
!python OpenNMT-tf/third_party/apply_bpe.py -c data/fa.code -i /content/data/train.fa -o /content/data/train.fa.bpe
!python OpenNMT-tf/third_party/apply_bpe.py -c data/fa.code -i /content/data/valid.fa -o /content/data/valid.fa.bpe

In [ ]:
!onmt-build-vocab --size 50000 --save_vocab /content/data/en-vocab.txt /content/data/train.en.bpe

In [ ]:
!onmt-build-vocab --size 50000 --save_vocab /content/data/fa-vocab.txt /content/data/train.fa.bpe

In [ ]:
#کپی کردن فایل کانفیگ در مسیر پردازش
!cp /content/drive/MyDrive/NLP/CA05/data.yml /content/data.yml

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
#شروع آموزش
!onmt-main train_and_eval --model_type Transformer --config data.yml --num_gpus 1








2022-06-05 14:56:06.596873: W tensorflow/python/util/util.cc:299] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.



INFO:tensorflow:Using parameters:
data:
  eval_features_file: /content/data/valid.en.bpe
  eval_labels_file: /content/data/valid.fa.bpe
  source_words_vocabulary: /content/data/en-vocab.txt
  target_words_vocabulary: /content/data/fa-vocab.txt
  train_features_file: /content/data/train.en.bpe
  train_labels_file: /content/data/train.fa.bpe
eval:
  batch_size: 30
  eval_delay: 1800
  export_format: saved_model
  export_on_best: bleu
  exporters: last
  length_bucket_width: 5
  max_exports_to_keep: 15
  save_eval_predictions: true
  scorers: bleu, rouge, wer, ter, prf
  steps: 1000
infer:
  batch_size: 128
  bucket_width: null
model_dir: /content/drive/MyDrive/NLP/CA05/run/
params:
  beam_width: 5
  clip_gradients: 5.0
  decay_params:
    decay_rate: 0.7
    decay_steps: 100000
  decay_type: exponentia

In [ ]:
#تا گام 17500 در این داکیومنت موجود است و ادامه فرآیند در فایل بعدی می‌باشد
#به دلیل طولانی شدن مدت پردازش و ارائه نکردن پردازشگر گرافیکی توسط گوگل در کولب

In [ ]:
# !cp -R /content/drive/MyDrive/NLP/CA05/run /content/drive/MyDrive/NLP/run

In [ ]:
!du -sh /content/drive/MyDrive/NLP/CA05/run

1.9G	/content/drive/MyDrive/NLP/CA05/run
